## LIDAR Handling

In this tutorial, I will show you how to run the LIDAR using the rplidar library.  
To begin with, you need to install the library using the command `pip install rplidar-roboticia`.  
You can do this in the terminal or by removing the '#' symbol from the block below.

In [ ]:
#!pip install rplidar-roboticia

Now you need to locate which port the LIDAR is on. If you are using the Jetson Nano AI Kit, it should be `/dev/ttyACM1`.   
Check if the library correctly detects the LIDAR by running the following block.

**Important - make sure the LIDAR is powered on, otherwise, it won't be detected!**

In [ ]:
%matplotlib inline

from rplidar import RPLidar
PORT_NAME = '/dev/ttyACM1'

lidar = RPLidar(PORT_NAME)
info = lidar.get_info()
if type(info) == type('string'):
    raise Exception("Buffer jest pełen, spróbuj inny port")
health = lidar.get_health()
for i in info:
    print(i,"-", info[i])
print(f'health - {health[0]}')

lidar.stop()
lidar.stop_motor()

    

---
The LIDAR should stop, and the block should return a similar result:

```
model - 24
firmware - (1, 29)
hardware - 7
serialnumber - BC91ED93C0EA98C7A0E69BF52C4F4560
health - Good
```

If you encounter an error like:

```
FileNotFoundError: [Errno 2] No such file or directory: '/dev/ttyUSB1'
```

Try using a different port.

---

Now let's test if the LIDAR is correctly collecting measurements. Run the next block!

In [ ]:
lidar.start_motor()

for i, scan in enumerate(lidar.iter_scans()):
    print('%d: Got %d measurments' % (i, len(scan)))
    if i > 8:
        break

lidar.stop_motor()

---
The LIDAR should spin, and the function should return a similar result:

```
0: Got 127 measurements
1: Got 171 measurements
2: Got 173 measurements
3: Got 167 measurements
4: Got 172 measurements
5: Got 174 measurements
6: Got 175 measurements
7: Got 167 measurements
8: Got 172 measurements
9: Got 168 measurements
```

This means everything is working correctly!

---

Now, let's move on to creating a simple map of the environment.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
from IPython.display import display, clear_output

def run_lidar():
    lidar = RPLidar(PORT_NAME)
    
    # wyczyszczenie danych pozostałych po poprzednim skanie
    lidar.clean_input()
    
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.set_rmax(4000)
    ax.grid(True)
    
    try:
        print("Starting to collect data...")
        
        while True:
            start_time = time.time()
            scans = []
            
            lidar.start_motor()
            for scan in lidar.iter_scans(max_buf_meas=5000):
                scans.append(scan)
                if time.time() - start_time > 2:  # Collect data for 2 seconds
                    break

            # zatrzymanie lasera na czas drukowania wykresu
            lidar.stop()
            
            # wyczyść zawartość wykresu
            ax.clear()
            ax.set_rmax(4000)
            ax.grid(True)
            
            # nanieś dane na wykres
            for scan in scans:
                angles = [item[1] for item in scan]
                distances = [item[2] for item in scan]
                ax.scatter(np.radians(angles), distances, c='r', s=5)
            
            clear_output()
            display(fig) # wyświetlenie outputu
            
            
    except Exception as e:
        print(f"An error occurred: {e}")
    except KeyboardInterrupt:
        print("Stopping...")
        lidar.stop()
        lidar.stop_motor()

if __name__ == '__main__':
    run_lidar()